<a href="https://colab.research.google.com/github/YokoyamaLab/PythonBasics/blob/main/day05_02Dropdowns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Day05 資料02 参考情報

## ドロップダウンメニューの作り方

このノートでは、みんなで実装した顔のパーツからドロップダウンメニューを作り、選んだパーツのクラスを返す方法を示します。

ドロップダウンが変更される度に```def on_value_change(change):```関数が呼ばれ、そのタイミングで全てのドロップダウンが何を選択しているかを取得すれば、「**今選ばれている顔のパーツ**」がわかります。

In [ ]:
# Google Driveに接続する
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/Shareddrives/2023-35A01／情報科学 c/lib

# UIをColab上で実現するモジュールを読みこむ
import ipywidgets as widgets
from IPython.display import display
# 元となるクラスをGoogle Driveから読みこむ
from face_component import *
# 宿題でみんなが提出したクラスをGoogle Driveから読みこむ
from face_homework import *
# その他必要なクラスを読みこむ
from PIL import Image, ImageDraw, ImageFont
import math,sys,inspect

# パーツのベースクラスをリストにしておく
component_base_classes = [Face, Eyes, Nose, Ears, Mouth, Hair]

# パーツを格納する辞書型変数を作る
components = {}
for base in component_base_classes:
  components[base.__name__] = [] # base.__name__はbaseのクラス名を文字列で得る

# みんなの実装したクラスを分類する
for name, component_class in inspect.getmembers(sys.modules[__name__]):
  if inspect.isclass(component_class) and issubclass(component_class,Component) and component_class != Component:
    types = list(filter(lambda x: issubclass(component_class,x), component_base_classes))
    if(len(types) > 0):
      components[types[0].__name__].append(component_class)
    else:
      print('Error: 認識していないクラスを継承しています')

# 分類されているか確認したい場合は以下の行のコメントを外して実行する
# print(components)

# 値が変更された時に呼ばれる関数を作っておく
def on_value_change(change):
  # 何が選ばれたか見たかったら以下の行のコメントを外す
  # print('【' + change['old']('red').feature + '】から【' + change['new']('red').feature + '】に変更されました。')
  # (何が変更されたかに関係なく、)全てdropdownの選ばれている値を取得する
  for key, dropdown in dropdowns.items(): # 辞書型をfor文で走査する
    # dropdown.valueは現在選ばれている値(今回の場合は顔のパーツクラス)
    print(dropdown.value)


# パーツ毎のドロップダウンリストを作る
dropdowns = {}
for base in component_base_classes:
  component_name = base.__name__
  # Dropdownには辞書型(key=feature/value=クラスそのもの)を格納(keyが表示される)
  opts = {}
  for idx,component in enumerate(components[component_name]):
    # もし同じfeatureのクラスがすでに存在していた場合は判別のためにクラス名を末尾に付与する
    if component('red').feature in opts:
      opts[component('red').feature + '(' + component.__name__ + ')'] = component
    else:
      opts[component('red').feature] = component
  dropdowns[component_name] = widgets.Dropdown(
    options = opts,                        # 選択肢のリスト
    description = component_name + ": ",   # ラベル
  )
  # Dropdownの選択が変わった時にon_value_change関数を呼び出し、選択肢のvalue(classそのもの)を返す
  dropdowns[component_name].observe(on_value_change, names='value')
  # Dropdownを表示する
  display(dropdowns[component_name])